# 4-3 Lane Filter and Car Commands


In [ ]:
import numpy as np
from scipy.stats import multivariate_normal, entropy
from scipy.ndimage.filters import gaussian_filter
from math import floor, atan2, pi, cos, sin, sqrt
import time
from matplotlib import pyplot as plt

### Environment Setup

In [ ]:
# constant
WHITE = 0
YELLOW = 1
RED = 2

lanewidth = 0.4
linewidth_white = 0.04
linewidth_yellow = 0.02


## Generate Vote from Line Segments

#### Setup a line segment
* left edge of white lane
* right edge of white lane
* left edge of yellow lane
* right edge of white lane

In [ ]:

# right edge of white lane
#p1 = np.array([0.8, 0.24])
#p2 = np.array([0.4, 0.24])
p1 = np.array([lines_normalized[0][0],lines_normalized[0][1]])
p2 = np.array([lines_normalized[0][2],lines_normalized[0][3]]) 
seg_color = YELLOW

# left edge of white lane
#p1 = np.array([0.4, 0.2])
#p2 = np.array([0.8, 0.2])
#seg_color = WHITE

#plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'ro')
#plt.plot([p1[0], p2[0]], [p1[1], p2[1]])
#plt.ylabel('y')
#plt.axis([0, 5, 0, 5])
#plt.show()

#### compute d_i, phi_i, l_i

In [ ]:
t_hat = (p2-p1)/np.linalg.norm(p2-p1)
n_hat = np.array([-t_hat[1],t_hat[0]])
d1 = np.inner(n_hat,p1)
d2 = np.inner(n_hat,p2)
l1 = np.inner(t_hat,p1)
l2 = np.inner(t_hat,p2)

print (d1, d2, l1, l2)

if (l1 < 0):
    l1 = -l1;
if (l2 < 0):
    l2 = -l2;
l_i = (l1+l2)/2
d_i = (d1+d2)/2
phi_i = np.arcsin(t_hat[1])
if seg_color == WHITE: # right lane is white
    if(p1[0] > p2[0]): # right edge of white lane
        d_i = d_i - linewidth_white
        print ('right edge of white lane')
    else: # left edge of white lane
        d_i = - d_i
        phi_i = -phi_i
        print ('left edge of white lane')
    d_i = d_i - lanewidth/2

elif seg_color == YELLOW: # left lane is yellow
    if (p2[0] > p1[0]): # left edge of yellow lane
        d_i = d_i - linewidth_yellow
        phi_i = -phi_i
        print ('right edge of yellow lane')
    else: # right edge of white lane
        d_i = -d_i
        print ('right edge of yellow lane')
    d_i =  lanewidth/2 - d_i

    
print (d_i, phi_i, l_i) 

### Measurement Likelihood

In [ ]:
# initialize measurement likelihood

d_min = -0.7
d_max = 0.5
delta_d = 0.02
phi_min = -pi/2
phi_max = pi/2
delta_phi = 0.02
d, phi = np.mgrid[d_min:d_max:delta_d, phi_min:phi_max:delta_phi]

measurement_likelihood = np.zeros(d.shape)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(measurement_likelihood, interpolation='nearest')
fig.colorbar(cax)
plt.ylabel('phi')
plt.xlabel('d')
#ax.set_xticklabels(['']+alpha)
#ax.set_yticklabels(['']+alpha)
plt.show()



In [ ]:
i = floor((d_i - d_min)/delta_d)
j = floor((phi_i - phi_min)/delta_phi)
measurement_likelihood[i,j] = measurement_likelihood[i,j] +  1/(l_i)

print (i, j)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(measurement_likelihood, interpolation='nearest')
fig.colorbar(cax)
plt.ylabel('phi')
plt.xlabel('d')
#ax.set_xticklabels(['']+alpha)
#ax.set_yticklabels(['']+alpha)
plt.show()

## Car Command

### Import Packages

In [ ]:
import numpy as np
import scipy as sp
import cv2
import time

from matplotlib import pyplot as plt
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels

### ROS Setup

In [ ]:
import sys
# rospy
sys.path.insert(0, '/opt/ros/indigo/lib/python2.7/dist-packages')
# rospkg
sys.path.insert(0, '/usr/lib/python2.7/dist-packages/')

# duckietown_msgs
duckietown_root = '../../'  # this file should be run from {duckietown_root}/turorials/python (otherwise change this line)
sys.path.insert(0, duckietown_root + 'catkin_ws/devel/lib/python2.7/dist-packages')

import rospy
from duckietown_msgs.msg import  Twist2DStamped


### ROS Node

You need to open a terminal in Duckiebot and run 
$ roscore

In [ ]:
rospy.init_node("joystick_jupyter",anonymous=False)
#please replace "wama" with your duckiebot name
pub_car_cmd = rospy.Publisher("/buyme/joystick_jupyter/car_cmd",Twist2DStamped,queue_size=1)


### Define a function for publishing car command 

In [ ]:
def car_command(v, omega, duration):
# Send stop command
    car_control_msg = Twist2DStamped()
    car_control_msg.v = v 
    car_control_msg.omega = omega 
    pub_car_cmd.publish(car_control_msg)
    rospy.sleep(duration)
    #rospy.loginfo("Shutdown")
    car_control_msg.v = 0.0 
    car_control_msg.omega = 0.0 
    pub_car_cmd.publish(car_control_msg)   

### Take a look at the ROS Topics

In [ ]:
%%bash
rostopic list

### Forward (F), Turn Left (L), or Turn Right (R)

Send commands and calibrate your duckiebot

### Ex1: Forward 0.5 Tile Width

In [ ]:
car_command(0.5, 0, 0.75)

### EX2: Turn 45 or 90 Degrees

In [ ]:
car_command(0.2, 4, 1.005)

### Setup a Switch for concat of primitives

In [ ]:
class switch(object):
    def __init__(self, value):
        self.value = value
        self.fall = False
 
    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration
     
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5, see below
            self.fall = True
            return True
        else:
            return False

def motion_concat(concat):
    for i in range(len(concat)):
        primitives = concat[i]
        for case in switch(primitives):
            if case('S'):
                car_command(0.5, 0, 0.25)
                break
            if case('L'):
                car_command(0.2, 4, 0.82)
                break
            if case('R'):
                car_command(0.2, -4, 0.78)
                break
            if case('B'):
                car_command(-0.4, 0, 0.5)
                break

### Ex3: Overtaking

In [ ]:
overtaking = "LSRSSSSRSLSS"
motion_concat(overtaking)

## Advanced: Bayes' Filter